In [59]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer , sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
lem = WordNetLemmatizer()

In [60]:
input_df = pd.read_excel('/content/Input.xlsx')
input_df.head(5)

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...


In [61]:
URLs = input_df['URL']
URLs

0      https://insights.blackcoffer.com/ai-in-healthc...
1      https://insights.blackcoffer.com/what-if-the-c...
2      https://insights.blackcoffer.com/what-jobs-wil...
3      https://insights.blackcoffer.com/will-machine-...
4      https://insights.blackcoffer.com/will-ai-repla...
                             ...                        
109    https://insights.blackcoffer.com/blockchain-fo...
110    https://insights.blackcoffer.com/the-future-of...
111    https://insights.blackcoffer.com/big-data-anal...
112    https://insights.blackcoffer.com/business-anal...
113    https://insights.blackcoffer.com/challenges-an...
Name: URL, Length: 114, dtype: object

In [62]:
# Loading positive words
with open('/content/positive-words.txt','r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')

# Loading negative words
with open('/content/negative-words.txt','r' ,  encoding="ISO-8859-1") as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

#Loading stop words dictionary for removing stop words
# GENERIC
with open('/content/StopWords_Generic.txt','r' ,  encoding="ISO-8859-1") as genericfile:
    genericword=genericfile.read().lower()
GenericWordList=genericword.split('\n')

# GENERIC LONG
with open('/content/StopWords_GenericLong.txt','r' ,  encoding="ISO-8859-1") as genericlongfile:
    genericlongword=genericlongfile.read().lower()
GenericLongWordList=genericlongword.split('\n')

# NAMES
with open('/content/StopWords_Names.txt','r' ,  encoding="ISO-8859-1") as namefile:
    names=namefile.read().lower()
nameList=names.split('\n')

# GEOGRAPHIC
with open('/content/StopWords_Geographic.txt','r' ,  encoding="ISO-8859-1") as geofile:
    geographics=geofile.read().lower()
geographicList=geographics.split('\n')

# GENERIC DATES
with open('/content/StopWords_DatesandNumbers.txt','r' ,  encoding="ISO-8859-1") as datefile:
    date=datefile.read().lower()
dateList=date.split('\n')

# CURRENCIES
with open('/content/StopWords_Currencies.txt','r' ,  encoding="ISO-8859-1") as currenciesfile:
    currencies=currenciesfile.read().lower()
currenciesList=currencies.split('\n')

# AUDITOR
with open('/content/StopWords_Auditor.txt','r' ,  encoding="ISO-8859-1") as auditfile:
    audit=auditfile.read().lower()
auditList=audit.split('\n')
# with open(stopWordsFile ,'r') as stop_words:
#     stopWords = stop_words.read().lower()
# stopWordList = stopWords.split('\n')
# stopWordList[-1:] = []

In [63]:
def ProperNounExtractor(text):
    count = 0
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged = nltk.pos_tag(words)
        for (word, tag) in tagged:
            if tag == 'PRP': # If the word is a proper noun
                count = count + 1 
    return(count)


def syllable_count(content):
    word=content.replace(' ','')
    syllable_count=0
    for w in word:
        if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
    syllable_per_word = syllable_count/len(content.split())
    return syllable_per_word

In [64]:
def get_article_names(urls):
  titles = []
  for i in range (len(urls)):
    title = urls[i]
    title_clean = title[title.index( "m/" ) + 2 :-1]. replace('-' , ' ')
    titles.append(title_clean)
  return titles

urls_title = get_article_names(URLs)
urls_title

['ai in healthcare to improve patient outcomes',
 'what if the creation is taking over the creator',
 'what jobs will robots take from humans in the future',
 'will machine replace the human in the future of work',
 'will ai replace us or work with us',
 'man and machines together machines are more diligent than humans blackcoffe',
 'in future or in upcoming years humans and machines are going to work together in every field of work',
 'how neural networks can be applied in various areas in the future',
 'how machine learning will affect your business',
 'deep learning impact on areas of e learning',
 'how to protect future data and its privacy blackcoffer',
 'how machines ai automations and robo human are effective in finance and banking',
 'ai human robotics machine future planet blackcoffer thinking jobs workplace',
 'how ai will change the world blackcoffer',
 'future of work how ai has entered the workplace',
 'ai tool alexa google assistant finance banking tool future',
 'ai heal

In [65]:
#Now fetching the page content.
#Taking a URL for checking that logic works or not.

url = "https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/"

page=requests.get(url , headers={"User-Agent": "XY"})  
soup = BeautifulSoup(page.text , 'html.parser')
#get title
title = soup . find("h1",attrs = { 'class' : 'entry-title'}).get_text()

#get article text
text = soup . find(attrs = { 'class' : 'td-post-content'}).get_text()
# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

text

'Introduction\n“If anything kills over 10 million people in the next few decades, it will be a highly infectious virus rather than a war. Not missiles but microbes.” Bill Gates’s remarks at a TED conference in 2014, right after the world had avoided the Ebola outbreak. When the new, unprecedented, invisible virus hit us, it met an overwhelmed and unprepared healthcare system and oblivious population. This public health emergency demonstrated our lack of scientific consideration and underlined the alarming need for robust innovations in our health and medical facilities. For the past few years, artificial intelligence has proven to be of tangible potential in the healthcare sectors, clinical practices, translational medical and biomedical research.\nAfter the first case was detected in China on December 31st 2019, it was an AI program developed by BlueDot that alerted the world about the pandemic. It was quick to realise AI’s ability to analyse large chunks of data could help in detecti

In [66]:
#Some websites aren't getting the proper requests so i'm avoiding them.
contents =[]
for url in URLs:

  page=requests.get(url , headers={"User-Agent": "XY"})  
  soup = BeautifulSoup(page.text , 'html.parser')
  #print(url)
  #get title

  try:
    title = soup.find("h1",attrs = { 'class' : 'entry-title'}).get_text()
  except:
    print(url, "can't found")
    print("************** \n")

  #get article text
  try:
    text = soup . find(attrs = { 'class' : 'td-post-content'}).get_text()
  except:
    print(url, "can't found content")
    print("************ \n")
  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  contents.append(text)

https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/ can't found
************** 

https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/ can't found content
************ 

https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/ can't found
************** 

https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/ can't found content
************ 

https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/ can't found
************** 

https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/ can't found content
************ 



In [67]:
#Analysis of the functions
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in GenericWordList, tokens))
    filtered_words = list(filter(lambda token: token not in GenericLongWordList, tokens))
    filtered_words = list(filter(lambda token: token not in nameList, tokens))
    filtered_words = list(filter(lambda token: token not in geographicList, tokens))
    filtered_words = list(filter(lambda token: token not in dateList, tokens))
    filtered_words = list(filter(lambda token: token not in currenciesList, tokens))
    filtered_words = list(filter(lambda token: token not in auditList, tokens))
    return filtered_words

def positive_score(text):
    posword=0
    tokenphrase = tokenizer(text)
    for word in tokenphrase:
        if word in positiveWordList:
            posword+=1
    retpos = posword
    return retpos 

def negative_score(text):
    negword=0
    tokenphrase = tokenizer(text)
    for word in tokenphrase:
        if word in negativeWordList: 
            negword +=1
    retneg = negword 
    return retneg

def polarity_score(positive_score , negative_score):
    return (positive_score - negative_score)/((positive_score + negative_score) + 0.000001)

def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

def AverageSentenceLength (text):
    Wordcount = len(tokenizer(text))
    SentenceCount = len(sent_tokenize(text))
    if SentenceCount > 0: 
        Average_Sentence_Lenght = Wordcount / SentenceCount
    avg = Average_Sentence_Lenght
    return round(avg)

def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex

def subjectivity(pos,neg,total):
    return (pos + neg)/ ((total) + 0.000001)

In [68]:
 def avg_word_length(content):
    return len(content.replace(' ',''))/len(content.split())

In [69]:
def avg_words_per_sent(content):
    AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
    AVG_NUMBER_OF_WORDS_PER_SENTENCE=sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE)
    return AVG_NUMBER_OF_WORDS_PER_SENTENCE

In [70]:
df = pd.DataFrame({'title':urls_title,'text': contents})

In [71]:
#Installing these packages for further process
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [72]:
df["AverageSentenceLength"] = df["text"].apply(AverageSentenceLength)
df["percentage_complex_word"] = df["text"].apply(percentage_complex_word)
df["fog_index"] = np.vectorize(fog_index)(df['AverageSentenceLength'],df['percentage_complex_word'])
df["complex_word_count"] = df["text"].apply(complex_word_count)
# df["total word count"] = df["text"].apply(total_word_count)
df["total_word_count"] = df["text"].apply(total_word_count)
df["positive_score"] = df["text"].apply(positive_score)
df["negative_score"] = df["text"].apply(negative_score)
df["polarity_score"] = np.vectorize(polarity_score)(df['positive_score'],df['negative_score'])
df["subjectivity"] = np.vectorize(subjectivity)(df['positive_score'],df['negative_score'], df['total_word_count'])

In [73]:
df['avg_word_length']=df['text'].apply(avg_word_length)
df['personal_pronouns'] = df['text'].apply(ProperNounExtractor)
df['syllable_per_word'] = df['text'].apply(syllable_count)
df['avg_words_per_sent'] = df['text'].apply(avg_words_per_sent)

In [74]:
df.head(5)

,title,text,AverageSentenceLength,percentage_complex_word,fog_index,complex_word_count,total_word_count,positive_score,negative_score,polarity_score,subjectivity,avg_word_length,personal_pronouns,syllable_per_word,avg_words_per_sent
0,ai in healthcare to improve patient outcomes,Introduction\n“If anything kills over 10 milli...,24,0.290411,9.716164,530,1825,82,36,0.389831,0.064658,5.820112,22,2.248045,22.400000
1,what if the creation is taking over the creator,"Human minds, a fascination in itself carrying ...",18,0.171626,7.268651,248,1445,72,37,0.321101,0.075433,5.019135,52,1.946137,17.231707
2,what jobs will robots take from humans in the ...,Introduction\nAI is rapidly evolving in the em...,20,0.267638,8.107055,459,1715,74,36,0.345455,0.064140,5.556471,28,2.187647,20.000000
3,will machine replace the human in the future o...,“Anything that could give rise to smarter-than...,17,0.205314,6.882126,340,1656,86,28,0.508772,0.068841,4.938182,50,1.951515,17.197917
4,will ai replace us or work with us,“Machine intelligence is the last invention th...,23,0.209966,9.283987,375,1786,71,29,0.420000,0.055991,5.270270,53,2.015526,19.863636


In [75]:
df.to_excel('Output Data Structure.xlsx', encoding='utf-8',index=False)